In [ ]:
# Cell 1: Imports and Setup
import json
from multiprocessing import cpu_count

import numpy as np
import tensorflow as tf
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, BatchNormalization, Dense, Dropout
from tensorflow.keras.models import Sequential
from tqdm import tqdm

print("✅ Imports loaded successfully!")
print(f"📊 TensorFlow version: {tf.__version__}")
print(f"🔧 Available CPUs: {cpu_count()}")

In [ ]:
# Cell 2: Constants and Paths (Updated for 64 frames + ISLR)
# UPDATED: 64 frames for complete sign capture
TARGET_FRAMES = 64
N_FACE = 468
N_POSE = 33
N_LEFT_HAND = 21
N_RIGHT_HAND = 21
TOTAL_LANDMARKS = N_FACE + N_POSE + N_LEFT_HAND + N_RIGHT_HAND  # 543
INPUT_SHAPE = (TARGET_FRAMES, TOTAL_LANDMARKS * 3)  # (64, 1629)

# Dataset Paths
ISLR_TRAIN_PATH = "/kaggle/input/wlasl2000-landmarks/train.tfrecord"  # 23.36 GB
ISLR_VAL_PATH = "/kaggle/input/wlasl2000-landmarks/val.tfrecord"      # 7.96 GB
WLASL_MAPPING_PATH = "/kaggle/input/wlasl200-mapping/label_mapping.json"  # 100 words
ISLR_MAPPING_PATH = "/kaggle/input/wlasl2000-landmarks/sign_to_prediction_index_map.json"  # 250 words

print("✅ Configuration set:")
print(f"   Target frames: {TARGET_FRAMES}")
print(f"   Total landmarks: {TOTAL_LANDMARKS}")
print(f"   Input shape: {INPUT_SHAPE}")
print("   Expected dataset size: ~50,000 samples")

In [ ]:
# Cell 3: Load Label Mappings (ISLR 250 + WLASL 100)
# Load ISLR 250-word mapping
with open(ISLR_MAPPING_PATH, 'r') as f:
    islr_mapping = json.load(f)

# Load WLASL 100-word mapping  
with open(WLASL_MAPPING_PATH, 'r') as f:
    wlasl_mapping = json.load(f)

# Create combined mapping (prioritize WLASL 100 words)
target_words = set(wlasl_mapping['gloss_to_label'].keys())
islr_words = set(islr_mapping.keys())
common_words = target_words.intersection(islr_words)

print("✅ Label mappings loaded:")
print(f"   ISLR words: {len(islr_mapping)}")
print(f"   WLASL target words: {len(target_words)}")
print(f"   Common words: {len(common_words)}")
print(f"   Common words: {sorted(list(common_words))[:20]}...")  # Show first 20

# Create final mapping for common words only
final_gloss_to_label = {}
final_label_to_gloss = {}
label_idx = 0

for word in sorted(common_words):
    final_gloss_to_label[word] = label_idx
    final_label_to_gloss[str(label_idx)] = word
    label_idx += 1

num_classes = len(final_gloss_to_label)
print(f"\n🎯 Final model will train on {num_classes} common words")

In [ ]:
# Cell 4: TFRecord Parsing Functions
def parse_tfrecord_example(example_proto):
    """Parse a single TFRecord example"""
    # Define the feature description based on ISLR format
    feature_description = {
        'landmarks': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'sequence_id': tf.io.FixedLenFeature([], tf.string),
    }
    
    parsed_features = tf.io.parse_single_example(example_proto, feature_description)
    
    # Decode landmarks
    landmarks = tf.io.decode_raw(parsed_features['landmarks'], tf.float32)
    landmarks = tf.reshape(landmarks, [-1, TOTAL_LANDMARKS, 3])  # [frames, 543, 3]
    
    label = parsed_features['label']
    sequence_id = parsed_features['sequence_id']
    
    return landmarks, label, sequence_id

def process_landmarks_to_target_frames(landmarks, target_frames=TARGET_FRAMES):
    """Resample landmarks to target frame count"""
    current_frames = tf.shape(landmarks)[0]
    
    # If already correct length, return as-is
    if current_frames == target_frames:
        return tf.reshape(landmarks, [target_frames, -1])  # Flatten to (64, 1629)
    
    # Resample using linear interpolation
    # Take first 64 frames if longer, or repeat if shorter
    if current_frames >= target_frames:
        # Take evenly spaced frames
        indices = tf.linspace(0.0, tf.cast(current_frames-1, tf.float32), target_frames)
        indices = tf.cast(indices, tf.int32)
        landmarks = tf.gather(landmarks, indices)
    else:
        # Pad with last frame if too short
        padding = target_frames - current_frames
        last_frame = landmarks[-1:]
        padding_frames = tf.tile(last_frame, [padding, 1, 1])
        landmarks = tf.concat([landmarks, padding_frames], axis=0)
    
    return tf.reshape(landmarks, [target_frames, -1])  # Flatten to (64, 1629)

print("✅ TFRecord parsing functions defined")

In [ ]:
# Cell 5: Load ISLR Dataset (This will take 10-15 minutes)
def load_islr_dataset(tfrecord_path, islr_to_final_mapping, max_samples=None):
    """Load and process ISLR TFRecord dataset"""
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    
    X_data = []
    y_data = []
    skipped = 0
    
    print(f"📦 Loading from {tfrecord_path}...")
    
    for i, record in enumerate(tqdm(dataset)):
        if max_samples and i >= max_samples:
            break
            
        try:
            landmarks, label, sequence_id = parse_tfrecord_example(record)
            
            # Convert TF tensors to numpy
            landmarks_np = landmarks.numpy()
            label_np = int(label.numpy())
            
            # Map ISLR label to word
            islr_word = None
            for word, islr_label in islr_mapping.items():
                if islr_label == label_np:
                    islr_word = word
                    break
            
            # Check if word is in our target set
            if islr_word and islr_word in final_gloss_to_label:
                # Process landmarks to 64 frames
                processed_landmarks = process_landmarks_to_target_frames(
                    tf.constant(landmarks_np)
                ).numpy()
                
                final_label = final_gloss_to_label[islr_word]
                
                X_data.append(processed_landmarks)
                y_data.append(final_label)
            else:
                skipped += 1
                
        except Exception:
            skipped += 1
            continue
    
    print(f"   Loaded: {len(X_data)} samples, Skipped: {skipped}")
    return np.array(X_data, dtype=np.float32), np.array(y_data, dtype=np.int32)

# Load training data (this takes time!)
print("🚀 Loading ISLR training data (this will take 10-15 minutes)...")
X_train_islr, y_train_islr = load_islr_dataset(ISLR_TRAIN_PATH, islr_mapping)

print("🚀 Loading ISLR validation data...")
X_val_islr, y_val_islr = load_islr_dataset(ISLR_VAL_PATH, islr_mapping)

print("\n✅ ISLR Dataset loaded:")
print(f"   Train: {len(X_train_islr)} samples, shape: {X_train_islr.shape}")
print(f"   Val: {len(X_val_islr)} samples, shape: {X_val_islr.shape}")
print(f"   Total: {len(X_train_islr) + len(X_val_islr)} samples")

# Combine train and val for final training
X_combined = np.concatenate([X_train_islr, X_val_islr], axis=0)
y_combined = np.concatenate([y_train_islr, y_val_islr], axis=0)

print(f"\n🎯 Combined dataset: {len(X_combined)} samples for {num_classes} classes")
unique_labels, counts = np.unique(y_combined, return_counts=True)
print(f"   Samples per class: min={counts.min()}, max={counts.max()}, mean={counts.mean():.1f}")

In [ ]:
# Cell 6: Train/Val Split (80/20)
X_train, X_val, y_train, y_val = train_test_split(
    X_combined, y_combined, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_combined
)

print("✅ Final Train/Val split:")
print(f"   Train: {len(X_train)} samples ({len(X_train)/len(X_combined)*100:.1f}%)")
print(f"   Val: {len(X_val)} samples ({len(X_val)/len(X_combined)*100:.1f}%)")
print(f"   Input shape: {X_train.shape[1:]}")
print(f"   Classes: {num_classes}")

# Verify data quality
print("\n🔍 Data quality check:")
print(f"   Train data range: [{X_train.min():.3f}, {X_train.max():.3f}]")
print(f"   NaN values: {np.isnan(X_train).sum()}")
print(f"   Inf values: {np.isinf(X_train).sum()}")

In [ ]:
# Cell 7: Build Enhanced LSTM Model (Optimized for 64 frames)
model = Sequential([
    # First LSTM layer - extract temporal features
    LSTM(128, return_sequences=True, activation='tanh', input_shape=INPUT_SHAPE),
    BatchNormalization(),
    Dropout(0.3),
    
    # Second LSTM layer - deeper temporal understanding  
    LSTM(256, return_sequences=True, activation='tanh'),
    BatchNormalization(),
    Dropout(0.3),
    
    # Third LSTM layer - final temporal encoding
    LSTM(128, return_sequences=False, activation='tanh'),
    BatchNormalization(),
    Dropout(0.4),
    
    # Dense layers for classification
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(128, activation='relu'),
    Dropout(0.3),
    
    # Output layer
    Dense(num_classes, activation='softmax')
])

# Compile with optimized settings
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', 'top_k_categorical_accuracy']
)

print("✅ Enhanced LSTM Model built:")
print(f"   Input: {INPUT_SHAPE}")
print(f"   Output: {num_classes} classes")
print(f"   Parameters: {model.count_params():,}")
print(f"   Model size: ~{model.count_params() * 4 / 1024 / 1024:.1f} MB")

model.summary()

In [ ]:
# Cell 8: Train Model (This will take 45-90 minutes)
# Callbacks for optimal training
checkpoint = ModelCheckpoint(
    '/kaggle/working/lstm_islr_combined_best.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=8,
    min_lr=1e-7,
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=20,
    restore_best_weights=True,
    verbose=1,
    mode='max'
)

print("🚀 Starting training (estimated time: 45-90 minutes)...")
print(f"   Dataset size: {len(X_train):,} training samples")
print("   Batch size: 64")
print("   Max epochs: 150")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=150,
    batch_size=64,  # Larger batch for better stability
    callbacks=[checkpoint, reduce_lr, early_stop],
    verbose=1
)

print("\n✅ Training complete!")
print(f"   Total epochs: {len(history.history['loss'])}")
print(f"   Best val accuracy: {max(history.history['val_accuracy'])*100:.2f}%")

In [ ]:
# Cell 9: Evaluate Model Performance
# Load best model
best_model = tf.keras.models.load_model('/kaggle/working/lstm_islr_combined_best.h5')

# Evaluate on validation set
print("🔍 Evaluating model performance...")
val_loss, val_acc, val_top5 = best_model.evaluate(X_val, y_val, verbose=0)

# Get predictions
y_pred = best_model.predict(X_val, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate additional metrics
top3_acc = top_k_accuracy_score(y_val, y_pred, k=3)
top5_acc = top_k_accuracy_score(y_val, y_pred, k=5)

print("\n🎯 FINAL RESULTS:")
print(f"   📊 Validation Loss: {val_loss:.4f}")
print(f"   🎯 Validation Accuracy: {val_acc*100:.2f}%")
print(f"   🥉 Top-3 Accuracy: {top3_acc*100:.2f}%")
print(f"   🥇 Top-5 Accuracy: {top5_acc*100:.2f}%")

# Show sample predictions
print("\n🔍 Sample predictions (first 15):")
for i in range(min(15, len(y_val))):
    true_word = final_label_to_gloss[str(y_val[i])]
    pred_word = final_label_to_gloss[str(y_pred_classes[i])]
    confidence = y_pred[i][y_pred_classes[i]] * 100
    status = "✅" if y_val[i] == y_pred_classes[i] else "❌"
    print(f"   {status} True: {true_word:15s} | Pred: {pred_word:15s} ({confidence:.1f}%)")

# Per-class accuracy
print("\n📈 Per-class performance:")
unique_labels = np.unique(y_val)
class_accuracies = []
for label in unique_labels:
    mask = y_val == label
    if mask.sum() > 0:
        acc = (y_pred_classes[mask] == label).mean()
        word = final_label_to_gloss[str(label)]
        class_accuracies.append(acc)
        if acc < 0.5:  # Show poorly performing classes
            print(f"   ⚠️  {word:15s}: {acc*100:.1f}% ({mask.sum()} samples)")

print("\n📊 Overall class performance:")
print(f"   Best class accuracy: {max(class_accuracies)*100:.1f}%")
print(f"   Worst class accuracy: {min(class_accuracies)*100:.1f}%")
print(f"   Average class accuracy: {np.mean(class_accuracies)*100:.1f}%")

In [ ]:
# Cell 10: Save Model and Create Deployment Package
# Save model in multiple formats
print("💾 Saving model and creating deployment package...")

# Save Keras model
best_model.save('/kaggle/working/lstm_islr_combined_final.h5')
best_model.save('/kaggle/working/lstm_islr_combined_savedmodel')

# Create comprehensive label mapping
deployment_config = {
    'model_info': {
        'name': 'ISLR+WLASL Combined LSTM',
        'version': '1.0',
        'input_shape': list(INPUT_SHAPE),
        'target_frames': TARGET_FRAMES,
        'total_landmarks': TOTAL_LANDMARKS,
        'num_classes': num_classes,
        'training_samples': len(X_train),
        'validation_samples': len(X_val)
    },
    'performance': {
        'val_accuracy': float(val_acc),
        'val_loss': float(val_loss),
        'top3_accuracy': float(top3_acc),
        'top5_accuracy': float(top5_acc)
    },
    'labels': {
        'gloss_to_label': final_gloss_to_label,
        'label_to_gloss': final_label_to_gloss,
        'num_classes': num_classes
    },
    'preprocessing': {
        'target_frames': TARGET_FRAMES,
        'landmarks_per_frame': TOTAL_LANDMARKS,
        'coordinates_per_landmark': 3,
        'input_flattened': True
    }
}

# Save deployment config
with open('/kaggle/working/deployment_config.json', 'w') as f:
    json.dump(deployment_config, f, indent=2)

# Save training history
history_data = {
    'loss': history.history['loss'],
    'accuracy': history.history['accuracy'],
    'val_loss': history.history['val_loss'],
    'val_accuracy': history.history['val_accuracy']
}

with open('/kaggle/working/training_history.json', 'w') as f:
    json.dump(history_data, f, indent=2)

print("\n✅ DEPLOYMENT PACKAGE READY:")
print("   📦 lstm_islr_combined_final.h5 (Keras model)")
print("   📦 lstm_islr_combined_savedmodel/ (TensorFlow SavedModel)")
print("   📦 deployment_config.json (Complete configuration)")
print("   📦 training_history.json (Training metrics)")

print("\n🎯 MODEL SUMMARY:")
print(f"   🎪 Trained on: {len(X_train):,} samples")
print(f"   🎯 Accuracy: {val_acc*100:.1f}%")
print(f"   📚 Vocabulary: {num_classes} words")
print(f"   ⚡ Input: {TARGET_FRAMES} frames ({TARGET_FRAMES/30:.1f}s @ 30fps)")
print(f"   💾 Model size: ~{best_model.count_params() * 4 / 1024 / 1024:.1f} MB")

print("\n📥 NEXT STEPS:")
print("   1. Download all 4 files from /kaggle/working/")
print("   2. Use deployment_config.json for webcam inference setup")
print(f"   3. Expected real-world accuracy: {val_acc*0.8*100:.1f}-{val_acc*0.9*100:.1f}%")